# AJAY KUMAR ( DATA MINING ASSIGNMENT 4)

In [13]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import timeit
from numpy import linalg as LA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,f1_score
import seaborn as sns
from scipy.stats import multivariate_normal
from sklearn.model_selection import KFold 
%matplotlib

Using matplotlib backend: Qt5Agg


# Q12:QDA on Data-1&2

In [14]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

In [15]:
def train_test(data):
    trn,tst = train_test_split(data, test_size = 0.3,shuffle=True)
    return trn,tst

In [16]:
# data normalisetion

def min_max_norm(data,data_min,data_max):
    for column in data.columns[:-1]:
        data[column] = (data[column] - data_min[column])/ (data_max[column]-data_min[column])    
    return data


In [17]:
def z_score(data,mean,std):
    for column in data.columns[:-1]:
        data[column] = (data[column] - mean[column])/ (std[column])    
    return data


In [18]:
# data classify

def classify(data,classes):
    N,n=data.shape[0],data.shape[1]
    c,l=[],[]
    for i in classes:
        s=data[data[n-1]==i].drop(columns=(n-1),axis=1)
        s_len=len(s)
        c.append(s)
        l.append(s_len)
    return c,l

In [19]:
def cov_mat(data,mu):
    N,n=data.shape[0],data.shape[1]
    sig=np.zeros((n,n))
    for i in range(N):
        t=data.iloc[i]-mu
        sig=sig+np.outer(t,t)
    return sig/(N-1)


In [20]:
def predict(data,mu,sig,pi,noc):
    N,n=data.shape[0],data.shape[1]
    X=data.drop(columns=(n-1),axis=1)
    del_x=np.zeros([N,noc])
    for i in range(N):
        for j in range(noc):
            sig_inv=LA.inv(sig[j])
            det_sig=LA.det(sig[j])
            t1=np.dot(mu[j],np.dot(sig_inv,X.iloc[i]))
            t2=-0.5*np.dot(mu[j],np.dot(sig_inv,mu[j]))
            t3=np.log(pi[j])
            t4=-0.5*np.log(det_sig)
            t5=-0.5*np.dot(X.iloc[i],np.dot(sig_inv,X.iloc[i]))
            del_x[i][j]=t1+t2+t3+t4+t5
    return del_x


In [21]:
def Qlda(data,mu,sig,pi,noc):
    N,n=data.shape[0],data.shape[1]
    X=data.drop(columns=(n-1),axis=1)
    for j in range(1,noc):
        sig_inv_k=LA.inv(sig[j])
        sig_inv_l=LA.inv(sig[j-1])
        det_sig=LA.det(sig[j])/LA.det(sig[j-1])
        pi_kl=pi[j]/pi[j-1]
        t1=np.dot(mu[j],sig_inv_k)-np.dot(mu[j-1],sig_inv_l)
        t2=-0.5*(np.dot(mu[j],np.dot(sig_inv_k,mu[j]))-np.dot(mu[j-1],np.dot(sig_inv_l,mu[j-1])))
        t3=np.log(pi_kl)
        t4=-0.5*np.log(det_sig)
        t5=-0.5*(sig_inv_k-sig_inv_l)
        a=t5
        b=t1
        c=t2+t3+t4
    return a,b,c

In [22]:
# performance check using confusion matrix

def metrics(y_tst,y_p,classes):
    d = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
    tp=y_tst.value_counts()[classes[1]]
    tn=y_tst.value_counts()[classes[0]]
    fp=len(d[(d[0]==classes[1]) & (d[1]==classes[0])])
    fn=len(d[(d[0]==classes[0]) & (d[1]==classes[1])])
    accuracy =(tp+tn)/(tp+tn+fp+fn)
    sensitivity=tp/(tp+fn)
    specificity=tn/(tn+fp)
    precision=tp/(tp+fp)
    fmeasure=2*(precision*sensitivity)/(precision+sensitivity)
    return tp,tn,fp,fn,accuracy,sensitivity,specificity,precision,fmeasure


In [23]:
pd.set_option('display.max_rows',200)
# pd.set_option('display.max_columns',200)

# Reading the data
df=read_data('data1.csv')#(200*3)

# Dropping duplicate rows if any
# dup=df[df.duplicated()] #Duplicated rows
# print(dup)
df.drop_duplicates(keep='first',inplace=True)#(190*3)

# Dropping columns having NaN values, if any
df.dropna(axis=1,inplace=True)

# Dropping columns with all zeros
df=df.loc[:, (df != 0).any(axis=0)]

#Relabelling the columns 
df.columns = pd.RangeIndex(len(df.columns))  
N,n=df.shape[0],df.shape[1]
# print(N,n)

# Identifying the various classes and their lengths
classes=df[n-1].drop_duplicates().sort_values().reset_index(drop=True)
noc=len(classes)
# print(classes)

# Splitting into training and testing data
trn,tst=train_test(df)
                                                        
# Normalising the data using min_max method
data_min,data_max=df.min(),df.max()
trn=min_max_norm(trn.copy(),data_min,data_max)
tst=min_max_norm(tst.copy(),data_min,data_max)

# Using Z-score method
# mean=np.mean(df,axis=0)
# std=np.std(df,axis=0)
# df=z_score(df,mean,std)

# Classifying the data into classes
cl,l=classify(trn,classes) #Returns two lists for different classes and the number of elements in them
# print(cl)

# Finding the stats for individual classes
pi,mu,sig=[],[],[]
for i in range(noc):
    p=l[i]/trn.shape[0] 
    m=cl[i].mean()
    s=cov_mat(cl[i],m)
    pi.append(p)
    mu.append(m)
    sig.append(s)
print("pi,mu,sig",pi,mu,sig,sep='\n')

# Predicting using the model
d=pd.DataFrame(predict(tst,mu,sig,pi,noc))
d.columns=classes
y_p=d.idxmax(axis=1)
# print(y_p)
y_tst=tst.iloc[:,n-1]


# Final Dataframe for showing actual Vs Prediction
s = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
s.columns=['y_tst','y_p']
# print(s)

# Finding the parameters
e,f,g=Qlda(trn,mu,sig,pi,noc)
e=e.flatten()
# print(e,f,g)

x=tst.iloc[:,0:n-1]
hyp_plane_1=np.zeros(x.shape[0])
hyp_plane_2=np.zeros(x.shape[0])
for i in range(x.shape[0]):
    x1=x.iloc[i,0]
    a=e[3]
#     print(a)
    b=x1*(e[2]+e[1])+f[1]
#     print(b)
    c=f[0]*x1 + e[0]*pow(x1,2)+g
#     print(c)
    d=pow(b,2)-4*a*c
    hyp_plane_1[i]=(-b+pow(d,0.5))/(2*a)
    hyp_plane_2[i]=(-b-pow(d,0.5))/(2*a)

# Results
print("\n No.of Classes is", noc)
# print("\n The Quadratic Discriminant Function, del is\n", d)

# Performance Parameters
metrics=pd.DataFrame(metrics(y_tst,y_p,classes))
metrics.index=['tp','tn','fp','fn','accuracy','sensitivity','specificity','precision','fmeasure']
metrics.columns=['Number of']
print("\nThe performance metrics are \n\n",metrics)

# Plotting Starts
# Hyperplane
plt.figure(1,figsize=(12,10))
plt.ylim(-0.1, 1.1)
plt.scatter(cl[0][0], cl[0][1], color='r',label='Positive')
plt.scatter(cl[1][0], cl[1][1], color='g',label='Negative')
plt.scatter(x[0],hyp_plane_1,color='b')
plt.scatter(x[0],hyp_plane_2,color='b')
plt.title('Hyperplane')
plt.xlabel('X1')
plt.ylabel('X2')

# ROC Curve
plt.figure(2,figsize=(12,10))
plt.subplot(211)
fpr, tpr, threshold = roc_curve(y_tst, y_p)
plt.plot(fpr,tpr)

# Confusion Matrix
plt.subplot(212)
cm = confusion_matrix(y_tst, y_p)
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')


# Probability Density Function for class:-1
rv = multivariate_normal.pdf(cl[0],mu[0],sig[0])
x,y=np.meshgrid(cl[0][0],cl[0][1])
z=np.outer(rv,rv)
fig = plt.figure(3)
ax = fig.add_subplot(1, 2, 1, projection='3d')
surf=ax.plot_surface(x,y,z,cmap='viridis',cstride=1,rstride=1,antialiased=False,linewidth=0)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Multivariate Gaussian Distribution for Positive class')
ax.set_xlabel('X1')
ax.set_ylabel('X2')

# Probability Density Function for class:1
rv = multivariate_normal.pdf(cl[1],mu[1],sig[1])
x,y=np.meshgrid(cl[1][0],cl[1][1])
z=np.outer(rv,rv)
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(x,y,z,cmap='plasma',cstride=1,rstride=1)
ax.set_title('Multivariate Gaussian Distribution for Negative Class')
ax.set_xlabel('X1')
ax.set_ylabel('X2')

# Contour Plot for class:-1
fig = plt.figure(4)
ax=fig.add_subplot(121)
x,y=np.meshgrid(cl[0][0],cl[0][1])
rv = multivariate_normal(mu[0],sig[0])
pos = np.dstack((x, y))
ax.contourf(x, y, rv.pdf(pos))
ax.set_title('Filled Contour Plot for class:-1')
ax.set_xlabel('feature_x1')
ax.set_ylabel('feature_x2')

# Contour Plot for class:1
ax=fig.add_subplot(122)
x,y=np.meshgrid(cl[1][0],cl[1][1])
rv = multivariate_normal(mu[1],sig[1])
pos = np.dstack((x, y))
ax.contourf(x, y, rv.pdf(pos))
ax.set_title('Filled Contour Plot for class:1')
ax.set_xlabel('feature_x1')
ax.set_ylabel('feature_x2')
plt.show()


pi,mu,sig
[0.47368421052631576, 0.5263157894736842]
[0    0.481697
1    0.302559
dtype: float64, 0    0.48309
1    0.67930
dtype: float64]
[array([[ 0.11658502, -0.02211018],
       [-0.02211018,  0.05307654]]), array([[0.05708638, 0.02108162],
       [0.02108162, 0.04553831]])]

 No.of Classes is 2

The performance metrics are 

              Number of
tp           25.000000
tn           32.000000
fp            7.000000
fn            5.000000
accuracy      0.826087
sensitivity   0.833333
specificity   0.820513
precision     0.781250
fmeasure      0.806452


# Q13:QDA on Data-3

In [14]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df


In [15]:
def train_test(data):
    trn,tst = train_test_split(data, test_size = 0.3,shuffle=True)
    return trn,tst

In [16]:
# data normalisetion

def min_max_norm(data,data_min,data_max):
    for column in data.columns[:-1]:
        data[column] = (data[column] - data_min[column])/ (data_max[column]-data_min[column])    
    return data

In [17]:
def z_score(data,mean,std):
    for column in data.columns[:-1]:
        data[column] = (data[column] - mean[column])/ (std[column])    
    return data

In [18]:
# classify data

def classify(data,classes):
    N,n=data.shape[0],data.shape[1]
    c,l=[],[]
    for i in classes:
        s=data[data[n-1]==i].drop(columns=(n-1),axis=1)
        s_len=len(s)
        c.append(s)
        l.append(s_len)
    return c,l

In [19]:
# covariance matrix

def cov_mat(data,mu):
    N,n=data.shape[0],data.shape[1]
    sig=np.zeros((n,n))
    for i in range(N):
        t=data.iloc[i]-mu
        sig=sig+np.outer(t,t)
    return sig/(N-1)


In [20]:
def predict(data,mu,sig,pi,noc):
    N,n=data.shape[0],data.shape[1]
    X=data.drop(columns=(n-1),axis=1)
    del_x=np.zeros([N,noc])
    for i in range(N):
        for j in range(noc):
            sig_inv=LA.inv(sig[j])
            det_sig=LA.det(sig[j])
            t1=np.dot(mu[j],np.dot(sig_inv,X.iloc[i]))
            t2=-0.5*np.dot(mu[j],np.dot(sig_inv,mu[j]))
            t3=np.log(pi[j])
            t4=-0.5*np.log(det_sig)
            t5=-0.5*np.dot(X.iloc[i],np.dot(sig_inv,X.iloc[i]))
            del_x[i][j]=t1+t2+t3+t4+t5
    return del_x


In [21]:
def Qlda(data,mu,sig,pi,j):
    N,n=data.shape[0],data.shape[1]
    X=data.drop(columns=(n-1),axis=1)
    sig_inv_k=LA.inv(sig[j])
    sig_inv_l=LA.inv(sig[j-1])
    det_sig=LA.det(sig[j])/LA.det(sig[j-1])
    pi_kl=pi[j]/pi[j-1]
    t1=np.dot(mu[j],sig_inv_k)-np.dot(mu[j-1],sig_inv_l)
    t2=-0.5*(np.dot(mu[j],np.dot(sig_inv_k,mu[j]))-np.dot(mu[j-1],np.dot(sig_inv_l,mu[j-1])))
    t3=np.log(pi_kl)
    t4=-0.5*np.log(det_sig)
    t5=-0.5*(sig_inv_k-sig_inv_l)
    a=t5
    b=t1
    c=t2+t3+t4
    return a,b,c


In [22]:
# confusion matrix performance check

def metrics(y_tst,y_p,classes):
    d = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
    tp=y_tst.value_counts()[classes[1]]
    tn=y_tst.value_counts()[classes[0]]
    fp=len(d[(d[0]==classes[1]) & (d[1]==classes[0])])
    fn=len(d[(d[0]==classes[0]) & (d[1]==classes[1])])
    accuracy =(tp+tn)/(tp+tn+fp+fn)
    sensitivity=tp/(tp+fn)
    specificity=tn/(tn+fp)
    precision=tp/(tp+fp)
    fmeasure=2*(precision*sensitivity)/(precision+sensitivity)
    return tp,tn,fp,fn,accuracy,sensitivity,specificity,precision,fmeasure


In [23]:
pd.set_option('display.max_rows',200)
# pd.set_option('display.max_columns',200)

# Reading the data
df=read_data('data3.csv')#(200*3)

# Dropping duplicate rows if any
# dup=df[df.duplicated()] #Duplicated rows
# print(dup)
df.drop_duplicates(keep='first',inplace=True)#(190*3)

# Dropping columns having NaN values, if any
df.dropna(axis=1,inplace=True)

# Dropping columns with all zeros
df=df.loc[:, (df != 0).any(axis=0)]

#Relabelling the columns 
df.columns = pd.RangeIndex(len(df.columns))  
N,n=df.shape[0],df.shape[1]
# print(N,n)

# Identifying the various classes and their lengths
classes=df[n-1].drop_duplicates().sort_values().reset_index(drop=True)
noc=len(classes)
# print(classes)

# Splitting into training and testing data
trn,tst=train_test(df)
                                                        
# # Normalising the data using min_max method
# data_min,data_max=df.min(),df.max()
# trn=min_max_norm(trn.copy(),data_min,data_max)
# tst=min_max_norm(tst.copy(),data_min,data_max)

# Using Z-score method
mean=np.mean(df,axis=0)
std=np.std(df,axis=0)
df=z_score(df,mean,std)

# Classifying the data into classes
cl,l=classify(trn,classes) #Returns two lists for different classes and the number of elements in them
# print(l)

# Finding the stats for individual classes
pi,mu,sig=[],[],[]
for i in range(noc):
    p=l[i]/trn.shape[0] 
    m=cl[i].mean()
    s=cov_mat(cl[i],m)
    pi.append(p)
    mu.append(m)
    sig.append(s)
print("pi,mu,sig\n",*pi,*mu,*sig,sep='\n')

# Predicting using the model
d=pd.DataFrame(predict(tst,mu,sig,pi,noc))
d.columns=classes
y_p=d.idxmax(axis=1)
# print(y_p)
y_tst=tst.iloc[:,n-1]

# Final Dataframe for showing actual Vs Prediction
s = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
s.columns=['y_tst','y_p']
match=len(np.where(s.y_tst==s.y_p)[0])
accuracy=match/len(y_tst)

# Finding the parameters
x=tst.iloc[:,0:n-1]
hyp_plane=[]
for j in range(1,noc):
    hyp_plane_1=np.zeros(x.shape[0])
    hyp_plane_2=np.zeros(x.shape[0])
    e,f,g=Qlda(trn,mu,sig,pi,j)
    e=e.flatten()
#     print(e,f,g)
    for i in range(x.shape[0]):
        x1=x.iloc[i,0]
        a=e[3]
#         print(a)
        b=x1*(e[2]+e[1])+f[1]
    #     print(b)
        c=f[0]*x1 + e[0]*pow(x1,2)+g
    #     print(c)
        dis=pow(b,2)-4*a*c
        hyp_plane_1[i]=(-b+pow(dis,0.5))/(2*a)
        hyp_plane_2[i]=(-b-pow(dis,0.5))/(2*a)
    hyp_plane.append(hyp_plane_1)
    hyp_plane.append(hyp_plane_2)

# Results
print("\n No.of Classes is", noc)
print("Accuracy %=",accuracy*100)
# print("\nActual Vs Prediction is\n",s)
# print("\n The Quadratic Discriminant Function, del is\n", d)

# Plotting Starts
# Hyperplane
fig = plt.figure(1,figsize=(12,10))
plt.ylim(-15, 15)
plt.scatter(cl[0][0], cl[0][1], color='r',label='Class-1')
plt.scatter(cl[1][0], cl[1][1], color='g',label='Class-2')
plt.scatter(cl[2][0], cl[2][1], color='b',label='Class-3')
plt.scatter(x[0],hyp_plane[0],color='k')
# plt.scatter(x[0],hyp_plane[1],color='k')
# plt.scatter(x[0],hyp_plane[2],color='y')
plt.scatter(x[0],hyp_plane[3],color='y')
plt.title('Hyperplane')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.show()

pi,mu,sig

0.3380952380952381
0.32857142857142857
0.3333333333333333
0   -4.079516
1   -3.290216
2    3.133214
dtype: float64
0    4.567567
1    4.263159
2   -2.931937
dtype: float64
0    0.436242
1    0.067545
2   -0.136274
dtype: float64
[[16.18208565 10.70817422  4.4628919 ]
 [10.70817422 17.83191584  1.55417172]
 [ 4.4628919   1.55417172  8.50911288]]
[[ 7.95764841  0.05135189 -0.7750151 ]
 [ 0.05135189  4.1741899   0.21473632]
 [-0.7750151   0.21473632  3.38231092]]
[[ 9.35753498 -1.40279642 -0.44463834]
 [-1.40279642  8.10603858  1.18710371]
 [-0.44463834  1.18710371  6.30050048]]

 No.of Classes is 3
Accuracy %= 86.66666666666667


# Q14-Naive Bayes for Data-4:Categorical

In [4]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

def train_test(data):
    trn,tst = train_test_split(data, test_size = 0.3,shuffle=True)
    return trn,tst

def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

# Calculate P(X=x1|Y=y)P(X=x2|Y=y)...P(X=xn|Y=y) * P(Y=y) for all y and find the maximum
def naive_bayes_categorical(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):
                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        if np.argmax(post_prob)==0:
            Y_pred.append(-1)
        else:
            Y_pred.append(np.argmax(post_prob))

    return np.array(Y_pred)

# Reading the data
df=read_data('data4.csv')#(171*4)
N,n=df.shape[0],df.shape[1]
print(N,n)

# Splitting into training and testing data
trn,tst=train_test(df)

# Splitting testing data into X and Y
X_tst = tst.iloc[:,:-1].values
Y_tst = tst.iloc[:,-1].values
Y_pred = naive_bayes_categorical(trn, X=X_tst, Y=n-1)

# Results
print("Test Values",Y_tst,"Predicted Values",Y_pred, sep='\n')
s=(Y_tst==Y_pred).sum()
accuracy=s/Y_tst.shape[0]
print("Accuracy %=",accuracy*100)

171 4
Test Values
[ 1  1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1  1  1  1 -1  1
 -1  1  1 -1  1  1 -1  1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1  1 -1 -1  1
 -1 -1  1  1]
Predicted Values
[ 1  1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1  1  1  1 -1  1
 -1  1  1 -1  1  1 -1  1 -1  1  1 -1 -1 -1 -1  1  1 -1  1  1  1 -1 -1  1
 -1 -1  1 -1]
Accuracy %= 96.15384615384616


# Q14-Naive Bayes for Data-5

In [5]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

def train_test(data):
    trn,tst = train_test_split(data, test_size = 0.3,shuffle=True)
    return trn,tst

def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
#     feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

# Calculate P(X=x1|Y=y)P(X=x2|Y=y)...P(X=xn|Y=y) * P(Y=y) for all y and find the maximum
def naive_bayes_categorical(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):
                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]
        if np.argmax(post_prob)==0:
            Y_pred.append(-1)
        else:
            Y_pred.append(1)
    return np.array(Y_pred)

# Reading the data
df=read_data('data5.csv')#(171*4)
N,n=df.shape[0],df.shape[1]
print(N,n)

for i in range(n-1):
    df[i] = pd.cut(df[i].values, bins = 10, labels = [0,1,2,3,4,5,6,7,8,9])
# print(df)

# Splitting into training and testing data
trn,tst=train_test(df)

# Splitting testing data into X and Y
X_tst = tst.iloc[:,:-1].values
Y_tst = tst.iloc[:,-1].values
Y_pred = naive_bayes_categorical(trn, X=X_tst, Y=n-1)

# Results
print("Test Values",Y_tst,"Predicted Values",Y_pred, sep='\n')
s=(Y_tst==Y_pred).sum()
accuracy=s/Y_tst.shape[0]
print("Accuracy %=",accuracy*100)

218 4
Test Values
[ 1  1 -1  1  1 -1  1  1 -1  1  1  1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1
 -1  1 -1 -1 -1  1  1  1 -1  1 -1  1  1 -1  1  1 -1  1  1 -1  1 -1  1 -1
 -1 -1  1 -1  1 -1 -1  1  1 -1  1 -1 -1 -1 -1 -1  1  1]
Predicted Values
[ 1  1 -1  1 -1 -1  1  1 -1  1  1  1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1
 -1  1 -1 -1 -1  1  1  1 -1  1 -1 -1  1 -1  1  1 -1 -1  1 -1  1 -1  1 -1
 -1 -1  1 -1  1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1  1  1]
Accuracy %= 93.93939393939394


# Q15-Multinomial Naive Bayes

In [6]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

def train_test(data):
    trn,tst = train_test_split(data, test_size = 0.3,shuffle=True)
    return trn,tst

def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

def calculate_likelihood_categorical(df, feat_name, feat_val, Y, label):
#     feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

# Calculate P(X=x1|Y=y)P(X=x2|Y=y)...P(X=xn|Y=y) * P(Y=y) for all y and find the maximum
def naive_bayes_categorical(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):
                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])

        # calculate posterior probability (numerator only)
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]
        if np.argmax(post_prob)==0:
            Y_pred.append(0)
        else:
            Y_pred.append(1)
    return np.array(Y_pred)

# Reading the data
df=read_data('data6.csv')#(171*4)
N,n=df.shape[0],df.shape[1]
print(N,n)

for i in range(n-1):
    df[i] = pd.cut(df[i].values, bins = 10, labels = [0,1,2,3,4,5,6,7,8,9])
# print(df)

# Splitting into training and testing data
trn,tst=train_test(df)

# Splitting testing data into X and Y
X_tst = tst.iloc[:,:-1].values
Y_tst = tst.iloc[:,-1].values
Y_pred = naive_bayes_categorical(trn, X=X_tst, Y=n-1)

# Results
print("Test Values",Y_tst,"Predicted Values",Y_pred, sep='\n')
s=(Y_tst==Y_pred).sum()
accuracy=s/Y_tst.shape[0]
print("Accuracy %=",accuracy*100)


208 4
Test Values
[1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 1 1 0 0 0 1
 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1]
Predicted Values
[1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 1 1
 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1]
Accuracy %= 88.88888888888889


# Q16-QDA for Ionosphere Data(Binary Classification)

In [13]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df


def train_test(X,y):
    return train_test_split(X,y, test_size = 0.3,shuffle=True)

def z_score(data,mean,std):
    for column in data.columns:
        data[column] = (data[column] - mean[column])/ (std[column])    
    return data

def classify(X,y,classes):
    c,l=[],[]
    for i in classes:
        s=X[y==i]
        s_len=len(s)
        c.append(s)
        l.append(s_len)
    return c,l

def cov_mat(data,mu):
    N,n=data.shape[0],data.shape[1]
    sig=np.zeros((n,n))
    for i in range(N):
        t=data.iloc[i]-mu
        sig=sig+np.outer(t,t)
    return sig/N-1

def predict(X,mu,sig,pi,noc):
    N,n=X.shape[0],X.shape[1]
    del_x=np.zeros([N,noc])
    for i in range(N):
        for j in range(noc):
            sig_inv=LA.inv(sig[j])
            det_sig=abs(LA.det(sig[j]))
            t1=np.dot(mu[j],np.dot(sig_inv,X.iloc[i]))
            t2=-0.5*np.dot(mu[j],np.dot(sig_inv,mu[j]))
            t3=np.log(pi[j])
            t4=-0.5*np.log(det_sig)
            t5=-0.5*np.dot(X.iloc[i],np.dot(sig_inv,X.iloc[i]))
            del_x[i][j]=t1+t2+t3+t4+t5
    return del_x

# Reading the data
df=read_data('Ionosphere.data')#(351*35)
# print(df.head(50))

# Dropping columns having NaN values, if any
df.dropna(axis=1,inplace=True)

# Dropping columns with all zeros and low variance
df=df.loc[:, (df != 0).any(axis=0)]#(350*34)
N,n=df.shape[0],df.shape[1]
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
sel = VarianceThreshold(0.25)
X=pd.DataFrame(sel.fit_transform(X))

#Relabelling the columns 
X.columns = pd.RangeIndex(len(X.columns))  
N,n=df.shape[0],df.shape[1]

# Identifying the various classes and their lengths
classes=y.drop_duplicates().sort_values().reset_index(drop=True)
noc=len(classes)

#K-fold cross validation
k = 5
kf = KFold(n_splits=k, random_state=None) 
acc_score = []
 
for train_index , test_index in kf.split(X):
    X_trn , X_tst = X.iloc[train_index,:],X.iloc[test_index,:]
    y_trn , y_tst = y[train_index] , y[test_index]
    
    # Using Z-score method
    mean=np.mean(X_trn,axis=0)
    std=np.std(X_trn,axis=0)
    X_trn=z_score(X_trn.copy(),mean,std)
    X_tst=z_score(X_tst.copy(),mean,std)
     
    # Classifying the data into classes
    cl,l=classify(X_trn,y_trn,classes) #Returns two lists for different classes and the number of elements in them

    # Finding the stats for individual classes
    pi,mu,sig=[],[],[]
    for i in range(noc):
        p=l[i]/X_trn.shape[0] 
        m=cl[i].mean()
        s=cov_mat(cl[i],m)
        pi.append(p)
        mu.append(m)
        sig.append(s)

    # Predicting using the model
    d=pd.DataFrame(predict(X_tst,mu,sig,pi,noc))
    d.columns=classes
    y_p=d.idxmax(axis=1)

    # Final Dataframe for showing actual Vs Prediction
    s = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
    s.columns=['y_tst','y_p']
    match=len(np.where(s.y_tst==s.y_p)[0])
    accuracy=match*100/len(y_tst)
    acc_score.append(accuracy)

avg_acc_score = sum(acc_score)/k
print('\n \n \n Result of K-fold method') 
print('\n accuracy of each fold :{}'.format(acc_score))
print('\n Avg accuracy : {}'.format(avg_acc_score))

# Holdout Method
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
acc_score=[]
threshold=[0,0.1,0.2,.25]

for i in threshold:
    sel = VarianceThreshold(i)
    X=pd.DataFrame(sel.fit_transform(X))
    
    # Splitting into training and testing data
    X_trn,X_tst,y_trn,y_tst=train_test(X,y)

    # Using Z-score method
    mean=np.mean(X_trn,axis=0)
    std=np.std(X_trn,axis=0)
    X_trn=z_score(X_trn.copy(),mean,std)
    X_tst=z_score(X_tst.copy(),mean,std)

    #Relabelling the columns 
    X_trn.columns = pd.RangeIndex(len(X_trn.columns)) 
    X_tst.columns = pd.RangeIndex(len(X_tst.columns))
    
    # Classifying the data into classes
    cl,l=classify(X_trn,y_trn,classes) #Returns two lists for different classes and the number of elements in them

    # Finding the stats for individual classes
    pi,mu,sig=[],[],[]
    for i in range(noc):
        p=l[i]/X_trn.shape[0] 
        m=cl[i].mean()
        s=cov_mat(cl[i],m)
        pi.append(p)
        mu.append(m)
        sig.append(s)

    # Predicting using the model
    d=pd.DataFrame(predict(X_tst,mu,sig,pi,noc))
    d.columns=classes
    y_p=d.idxmax(axis=1)

    # Final Dataframe for showing actual Vs Prediction
    s = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
    s.columns=['y_tst','y_p']
    match=len(np.where(s.y_tst==s.y_p)[0])
    accuracy=match*100/len(y_tst)
    acc_score.append(accuracy)

avg_acc_score = sum(acc_score)/len(threshold)
print('\n \n \n Result of Holdout method') 
print('\n accuracy of each iteration :{}'.format(acc_score))
print('\n Avg accuracy : {}'.format(avg_acc_score))


 
 
 Result of K-fold method

 accuracy of each fold :[85.91549295774648, 92.85714285714286, 85.71428571428571, 91.42857142857143, 97.14285714285714]

 Avg accuracy : 90.61167002012073

 
 
 Result of Holdout method

 accuracy of each iteration :[87.73584905660377, 84.90566037735849, 90.56603773584905, 95.28301886792453]

 Avg accuracy : 89.62264150943395


# Q17-QDA for Abalone Data(Multiclass Classification)

In [10]:
def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

def train_test(X,y):
    return train_test_split(X,y, test_size = 0.3,shuffle=True)

def z_score(data,mean,std):
    for column in data.columns:
        data[column] = (data[column] - mean[column])/ (std[column])    
    return data

def classify(X,y,classes):
    c,l=[],[]
    for i in classes:
        s=X[y==i]
        s_len=len(s)
        c.append(s)
        l.append(s_len)
    return c,l

def cov_mat(data,mu):
    N,n=data.shape[0],data.shape[1]
    sig=np.identity(n)
    for i in range(N):
        t=data.iloc[i]-mu
        sig=sig+np.outer(t,t)
    return sig/N-1

def predict(X,mu,sig,pi,noc):
    N,n=X.shape[0],X.shape[1]
    del_x=np.zeros([N,noc])
    for i in range(N):
        for j in range(noc):
            if(pi[j]*100000>1000):
                sig_inv=LA.inv(sig[j])
                det_sig=abs(LA.det(sig[j]))
                t1=np.dot(mu[j],np.dot(sig_inv,X.iloc[i]))
                t2=-0.5*np.dot(mu[j],np.dot(sig_inv,mu[j]))
                t3=np.log(pi[j])
                t4=-0.5*np.log(det_sig)
                t5=-0.5*np.dot(X.iloc[i],np.dot(sig_inv,X.iloc[i]))
                del_x[i][j]=t1+t2+t3+t4+t5
    return del_x

# Reading the data
df=read_data('abalone.data')
# print(df.head(50))

# Dropping columns having NaN values, if any
df.dropna(axis=1,inplace=True)

# Dropping columns with all zeros and low variance
df=df.loc[:, (df != 0).any(axis=0)]
df[0].replace(to_replace=['M','F','I'],value=[-1,0,1],inplace=True)
N,n=df.shape[0],df.shape[1]

X=df.iloc[:,:-1]
y=df.iloc[:,-1]
# sel = VarianceThreshold(0.1)
# X=pd.DataFrame(sel.fit_transform(X))

#Relabelling the columns 
X.columns = pd.RangeIndex(len(X.columns))  
N,n=df.shape[0],df.shape[1]

# Identifying the various classes and their lengths
classes=y.drop_duplicates().sort_values().reset_index(drop=True)
noc=len(classes)
print(noc)

# Splitting into training and testing groups
X_trn,X_tst,y_trn,y_tst=train_test_split(X,y)
    
# Using Z-score method
mean=np.mean(X_trn,axis=0)
std=np.std(X_trn,axis=0)
X_trn=z_score(X_trn.copy(),mean,std)
X_tst=z_score(X_tst.copy(),mean,std)

# Classifying the data into classes
cl,l=classify(X_trn,y_trn,classes) #Returns two lists for different classes and the number of elements in them
print(l)

# Finding the stats for individual classes
pi,mu,sig=[],[],[]
for i in range(noc):
    p=l[i]/X_trn.shape[0] 
    m=cl[i].mean()
    s=cov_mat(cl[i],m)
    pi.append(p)
    mu.append(m)
    sig.append(s)
print(pi)

# Predicting using the model
d=pd.DataFrame(predict(X_tst,mu,sig,pi,noc))
d.columns=classes
y_p=d.idxmax(axis=1)

# Final Dataframe for showing actual Vs Prediction
s = pd.concat([y_tst.reset_index(drop=True), y_p.reset_index(drop=True)],axis=1,ignore_index=True)
s.columns=['y_tst','y_p']
match=len(np.where(s.y_tst==s.y_p)[0])
accuracy=match/len(y_tst)

# Results
print("\n No.of Classes is", noc)
print("Accuracy %=",accuracy*100)
# print("\nActual Vs Prediction is\n",s)
# print("\n The Quadratic Discriminant Function, del is\n", d)

28
[1, 1, 11, 49, 90, 189, 294, 417, 520, 476, 371, 195, 145, 96, 79, 52, 43, 32, 22, 23, 9, 4, 7, 2, 1, 1, 1, 1]
[0.00031928480204342275, 0.00031928480204342275, 0.00351213282247765, 0.015644955300127713, 0.028735632183908046, 0.0603448275862069, 0.09386973180076628, 0.1331417624521073, 0.16602809706257982, 0.15197956577266922, 0.11845466155810984, 0.06226053639846743, 0.046296296296296294, 0.03065134099616858, 0.025223499361430396, 0.016602809706257982, 0.013729246487867178, 0.010217113665389528, 0.0070242656449553, 0.007343550446998723, 0.0028735632183908046, 0.001277139208173691, 0.002234993614303959, 0.0006385696040868455, 0.00031928480204342275, 0.00031928480204342275, 0.00031928480204342275, 0.00031928480204342275]

 No.of Classes is 28
Accuracy %= 22.48803827751196


# Q17-Multinomial Naive Bayes : Abalone Data(Multiclass)

In [11]:
class NaiveBayes:
    
    def __init__(self, alpha=1.0):
        self.prior = None
        # Alpha is a value, usually 1, in order to avoid the situation of features with 0 occurances
        # totally canceling out the numerator.
        self.alpha = alpha
        self.col_probs = None
        
    def fit(self, X, y):
        feature_probs = {}
        prior_probs = {}
        lb=[]
        for row, label in zip(X, y):
            if label not in feature_probs:
                prior_probs[label] = 0
                feature_probs[label] = [self.alpha] * len(X[0])
                lb.append(label)
            prior_probs[label] += (1 / len(y))
            
            for feature_idx, feature in enumerate(row):
                feature_probs[label][feature_idx] += feature
        
        self.prior = prior_probs
        self.lb=lb

        for label, counts in feature_probs.items():
            for feature_idx, feature in enumerate(counts):
                feature_probs[label][feature_idx] = feature / sum(counts)
        self.feature_probs = feature_probs
        return self
    
    def predict(self,X):
        N,n=X.shape[0],X.shape[1]
        
        del_x=np.zeros([N,30])
        for i in range(N):
            for j in self.prior:
                t1=np.log(self.prior[j])
                t2=np.dot(X[i],np.log(self.feature_probs[j]))
                del_x[i][j]=t1+t2
        return del_x

def read_data(data):
    cwd=os.getcwd()
    path=os.path.join(cwd,'assign_4', data)
    df=pd.read_csv(path,delimiter=',',header=None,na_values='?')
    return df

def train_test(data):
    return train_test_split(data, test_size = 0.3,shuffle=True)


# Reading the data
df=read_data('abalone.data')

# Dropping columns having NaN values, if any
df.dropna(axis=1,inplace=True)

# Dropping columns with all zeros and low variance
df=df.loc[:, (df != 0).any(axis=0)]
df[0].replace(to_replace=['M','F','I'],value=[-1,0,1],inplace=True)
N,n=df.shape[0],df.shape[1]

for i in range(n-1):
    df[i] = pd.cut(df[i].values, bins = 10, labels = [0,1,2,3,4,5,6,7,8,9])
    
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

# Splitting testing data into X and Y
X_trn,X_tst,y_trn,y_tst=train_test_split(X,y)

# Identifying the various classes and their lengths
classes=df.iloc[:,-1].drop_duplicates().sort_values().reset_index(drop=True)
noc=len(classes)

clf = NaiveBayes()
clf.fit(X_trn, y_trn)
print(clf.prior)

# Predicting using the model
d=pd.DataFrame(clf.predict(X_tst))
c=d.loc[(d!=0).any(axis=1)]
print(c)
y_p=d.idxmax(axis=1)

# Final Dataframe for showing actual Vs Prediction
s = pd.concat([pd.DataFrame(y_tst), y_p.reset_index(drop=True),d.reset_index(drop=True)],axis=1,ignore_index=True)
match=len(np.where(s[0]==s[1])[0])
accuracy=match/len(y_tst)

# Results
print("\n No.of Classes is",noc )
print("Accuracy %=",accuracy*100)
# print("\nActual Vs Prediction is\n",s)

{8: 0.1340996168582374, 6: 0.06034482758620679, 11: 0.10855683269476334, 20: 0.0063856960408684525, 19: 0.00606641123882503, 10: 0.1542145593869738, 13: 0.051404853128991006, 12: 0.06864623243933574, 17: 0.015006385696040861, 23: 0.002554278416347382, 7: 0.08971902937420151, 9: 0.16762452107279804, 26: 0.00031928480204342275, 4: 0.012771392081736903, 15: 0.022988505747126464, 14: 0.030332056194125222, 5: 0.030012771392081798, 18: 0.01117496807151979, 16: 0.016283524904214555, 3: 0.004150702426564495, 21: 0.003831417624521072, 29: 0.00031928480204342275, 24: 0.0006385696040868455, 22: 0.0015964240102171138, 2: 0.00031928480204342275, 1: 0.00031928480204342275, 27: 0.00031928480204342275}
       0          1          2          3          4          5          6   \
0     0.0 -54.184969 -50.661394 -34.287339 -30.908763 -24.643178 -24.124265   
1     0.0 -54.126550 -50.603069 -35.856824 -32.827016 -25.004315 -25.461600   
2     0.0 -41.581927 -38.531603 -24.817541 -21.928496 -19.113532 -1